In [1]:
# import libraries and dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# create a client instance
mongo = MongoClient(port=27017)

In [3]:
# database variable assignment
uk_food = mongo['uk_food']

# db.collection variable assignment
establishments = uk_food['establishments']

In [14]:
# confirm database and collection
print(f'confirmation that uk_food database exists: {mongo.list_database_names()}')
print(f'confirmation that uk_food.establishments collection exists: {uk_food.list_collection_names()}')

confirmation that uk_food database exists: ['admin', 'config', 'local', 'met', 'myDatabase', 'uk_food']
confirmation that uk_food.establishments collection exists: ['establishments']


In [15]:
# pprint find_one establishment

pprint(establishments.find_one())

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66f9a3b5a32acdebf58d6c5c'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [16]:
# new halal restaurant {'BusinessName': 'Penang Flavours'}

new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('66f9ac6e6dc2638d9f44280d'), acknowledged=True)

In [17]:
# find the {'BusinessTypeID'} for {'BusinessType':'Restaurant/Cafe/Canteen'}

result = establishments.find_one({'BusinessType':'Restaurant/Cafe/Canteen'}, {'BusinessTypeID':1,'BusinessType':1})
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('66f9a3b5a32acdebf58d6c5c')}


In [18]:
# update the {'BusinessTypeID'} for the new restaurant {'BusinessName': 'Penang Flavours'} with the result found above

establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set': {'BusinessTypeID': result['BusinessTypeID']}}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [19]:
# check and return count for how many documents contain {'LocalAuthorityName': 'Dover'}

dover_count_before = establishments.count_documents({'LocalAuthorityName': 'Dover'})
print(f'Establishments in Dover before deletion: {dover_count_before}')

# remove all {'LocalAuthorityName': 'Dover'} documents
establishments.delete_many({'LocalAuthorityName': 'Dover'})

# check and return count for the updated amount of documents containing {'LocalAuthorityName': 'Dover'} --> answer should be 0
dover_count_after = establishments.count_documents({'LocalAuthorityName': 'Dover'})
print(f'Establishments in Dover after deletion: {dover_count_after}')


Establishments in Dover before deletion: 994
Establishments in Dover after deletion: 0


In [20]:
# update_many so that 'geocode.latitude' and 'geocode.longitude' are set to decimal numbers

establishments.update_many({}, [{'$set': {'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}])
establishments.update_many({}, [{'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}}}])


UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# based on assignment note that the rating value field also includes non-numeric values such as 'Pass' and others, so we will filter out all non-numeric values aka treating them as null

non_numeric_ratings = establishments.distinct('RatingValue', {'RatingValue': {'$not': {"$regex": "^[0-9]+$"}}})
print(non_numeric_ratings)

['Awaiting Inspection', 'AwaitingInspection', 'AwaitingPublication', 'Exempt', 'Pass']


In [22]:
# [starter code] Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

# update_many to convert RatingValues to integers

establishments.update_many({'RatingValue': {'$type': 'string'}}, [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}])

UpdateResult({'n': 34694, 'nModified': 34694, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [25]:
# check that RatingValue and geocodes are integers
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66f9a3b5a32acdebf58d6f43'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,
